<a href="https://colab.research.google.com/github/ramonVDAKKER/teaching-quantitative-finance/blob/main/notebooks/illustration_discrete_time_hedging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook illustration (discrete-time) delta-hedging and delta-gamma-hedging

## 0. Imports

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
%%capture
!git clone https://github.com/ramonVDAKKER/teaching-quantitative-finance
import os
os.chdir("teaching-quantitative-finance/notebooks")
from utils.black_scholes import BlackScholesOptionPrice

## 1. Example discrete-time delta-hedging

The setup is as follows:
*   we adopt the standard models for $B$ and $S$;
*   we consider a situation in which we are only allowed to trade on a discrete-time grid;
*   financial institution that has a position of $a$ put options with exercise price $K$ and maturity $T$ at $t=0$ (in case $a<0$ the institution sells (writes) the puts, in case $a>0$ the institution buys the options);
*   we will evaluate two strategies in this section:
    1.   No active risk management: the institution takes no actions apart from investing $-a p_0$, with $p_0$ the price of one put at $t=0$, in the money-market-account $B$. This implies that there are no net cashflows in the time interval $[0,T)$. And at maturity $T$ the loss equals $a\times (\max(K-S_T, 0) - p_0 \exp(rT))$ at maturity.
    2.   Delta-hedging: the institution trades in $S$ and $B$, at each point-in-time (on the discrete grid), such that a) the total portfolio (of the puts and the positions in $S$ and $B$) is delta-neutral and b) the rebalancing of the positions in $S$ and $B$ is budget-neutral.

*Remark:* Please recall that an implementation of 2) in continuous-time would imply that the total portfolio has value 0 at each point in time $t\in [0, T]$. This means that the institution would be able to take the position in the put (to serve its clients) without bearing any risk.




### Parameters

In [ ]:
# granularity time grid [number of points per unit-of-time [=1 year]]
num_time_steps_per_unit_of_time = 252

# parameters GBM for S
S_0 = 100
mu = 0.05
sigma = 0.20

# parameters B
B_0 = 1
r = 0.01

# put option specs
T = 1
K = 90
num_puts = - 1000 # negative=instution writes puts (for client who wants to buy put), positive=institution buys puts (for client who wants to write put)

### Helper Functions

First, we define some utility functions:

In [ ]:
def validate_parameters(K, T, S_0, sigma, r, num_time_steps_per_unit_of_time):
    """Validate input parameters."""
    if K <= 0:
        raise ValueError("Strike price K must be strictly positive")
    if T <= 0:
        raise ValueError("Time to maturity T must be strictly positive")
    if S_0 <= 0:
        raise ValueError("Initial stock price S_0 must be strictly positive")
    if sigma <= 0:
        raise ValueError("Volatility sigma must be strictly positive")
    if num_time_steps_per_unit_of_time < 1:
        raise ValueError("Number of time steps must be at least 1")

def initialize_portfolio(S_0: float, T: float, K: float, r: float, sigma: float, 
                        num_puts: int, B_0: float):
    """Initialize portfolio positions at t=0."""    
    put = BlackScholesOptionPrice(K, r, sigma)
    put_price_initial = put.price_put(current_stock_price=S_0, time_to_maturity=T)
    price_puts = num_puts * put_price_initial
    phi = -num_puts * put.delta_put(current_stock_price=S_0, time_to_maturity=T) # delta-neutral
    psi = -(price_puts + phi * S_0) / B_0
    
    return phi, psi, price_puts, put

def rebalance_portfolio(S: float, B: float, time_to_maturity: float, 
                       phi_prev: float, psi_prev: float, num_puts: int, 
                       put: BlackScholesOptionPrice):
    """Rebalance portfolio to maintain delta neutrality."""
    value = phi_prev * S + psi_prev * B
    price_puts = num_puts * put.price_put(current_stock_price=S, time_to_maturity=time_to_maturity)
    phi = -num_puts * put.delta_put(current_stock_price=S, time_to_maturity=time_to_maturity) # delta-neutral
    psi = (value - phi * S) / B # budget-neutral
    
    return phi, psi, price_puts, value

def initialize_portfolio_vectorized(S_0: float, T: float, K: float, r: float, sigma: float,
                                   num_puts: int, B_0: float, M: int, put: BlackScholesOptionPrice):
    """
    Initialize portfolio positions at t=0 for M simulations.
    
    Returns arrays of shape (M,) for initial positions.
    """
    put_price_initial = put.price_put(current_stock_price=S_0, time_to_maturity=T)
    price_puts = np.full(M, num_puts * put_price_initial)
    phi = np.full(M, -num_puts * put.delta_put(current_stock_price=S_0, time_to_maturity=T))
    psi = -(price_puts + phi * S_0) / B_0
    
    return phi, psi, price_puts

def rebalance_portfolio_vectorized(S, B, time_to_maturity: float,
                                  phi_prev, psi_prev, num_puts: int,
                                  put: BlackScholesOptionPrice):
    """
    Rebalance portfolio to maintain delta neutrality for vectorized simulations.
    
    S, B, phi_prev, psi_prev are arrays of shape (M,) representing M simulations.
    Returns updated positions for all M simulations.
    """
    value = phi_prev * S + psi_prev * B
    price_puts = num_puts * put.price_put(current_stock_price=S, time_to_maturity=time_to_maturity)
    phi = -num_puts * put.delta_put(current_stock_price=S, time_to_maturity=time_to_maturity)
    psi = (value - phi * S) / B
    
    return phi, psi, price_puts, value


### Main Delta Hedging Function

The following function implements the discrete-time delta hedging strategy as described above.

In [ ]:
def writing_put_option_delta_hedge_discrete_time(K: float, T: float, S_0: float, mu: float, sigma: float,
                                                 B_0: float, r: float, num_time_steps_per_unit_of_time: int,
                                                 num_puts: int):
    """Discrete-time delta hedging simulation for put options."""
    validate_parameters(K, T, S_0, sigma, r, num_time_steps_per_unit_of_time)
    
    num_time_steps_total = int(T * num_time_steps_per_unit_of_time)
    time_delta = T / num_time_steps_total
    
    # Initialize variables
    phi = np.zeros(num_time_steps_total + 1)
    psi = np.zeros(num_time_steps_total + 1)
    phi[-1] = np.nan
    psi[-1] = np.nan
    price_puts = np.zeros(num_time_steps_total + 1)
    S = np.zeros(num_time_steps_total + 1)
    S[0] = S_0
    B = np.zeros(num_time_steps_total + 1)
    B[0] = B_0
    total_portfolio_value = np.zeros(num_time_steps_total + 1)
    time = np.linspace(0, T, num_time_steps_total + 1)
    
    # Initialize portfolio using helper function
    phi[0], psi[0], price_puts[0], put = initialize_portfolio(S_0, T, K, r, sigma, num_puts, B_0)
    total_portfolio_value[0] = price_puts[0] + phi[0] * S[0] + psi[0] * B[0]  # should be 0 by construction
    
    # Iterate over discrete-time grid
    for k in range(1, num_time_steps_total + 1):
        # Update asset prices (exact simulation of GBM)
        B[k] = B[k - 1] * np.exp(r * time_delta)
        S[k] = S[k - 1] * np.exp((mu - 0.5 * sigma ** 2) * time_delta + 
                                  sigma * np.sqrt(time_delta) * norm.rvs())
        
        # At maturity
        if k == num_time_steps_total:
            value = phi[k - 1] * S[k] + psi[k - 1] * B[k]
            price_puts[k] = num_puts * np.maximum(K - S[k], 0)
            total_portfolio_value[k] = price_puts[k] + value
            break
        
        # Rebalance portfolio using helper function
        time_to_maturity = T - time[k]
        phi[k], psi[k], price_puts[k], value = rebalance_portfolio(
            S[k], B[k], time_to_maturity, phi[k-1], psi[k-1], num_puts, put
        )
        
        total_portfolio_value[k] = price_puts[k] + value
    
    return time, S, B, phi, psi, price_puts, total_portfolio_value


In [ ]:
time, S, B, phi, psi, price_puts, total_portfolio_value = writing_put_option_delta_hedge_discrete_time(K, T, S_0, mu, sigma,
                                                 B_0, r, num_time_steps_per_unit_of_time,
                                                num_puts
                                                 )
df = pd.DataFrame(data = np.array([time, S, B, phi, psi, price_puts, total_portfolio_value]).T, columns=["t", "S", "B",
                                "position S", "position B", "num_puts * put_price", "total_portfolio_value"])
fig, ax = plt.subplots(2, 3, figsize=(25, 10))
df.plot(x="t", y="B", title="path B", ax=ax[0, 0])
df.plot(x="t", y="S", title="path S (red=strike puts)", ax=ax[0, 1])
ax[0, 1].axhline(y=K, color="r")
df.plot(x="t", y="num_puts * put_price", title=f"value {num_puts} puts", ax=ax[0, 2])
df.iloc[:-1].plot(x="t", y="position B", title="path psi (position B)", ax=ax[1, 0])
df.iloc[:-1].plot(x="t", y="position S", title="path phi (position S)", ax=ax[1, 1])
df.plot(x="t", y="total_portfolio_value", title="mismatch", ax=ax[1, 2])

In [ ]:
print(f"Total value of {num_puts} puts at t=0: {price_puts[0]}")

Next we use Monte Carlo simulations (replications) to approximate the distribution of the total portfolio value (consisting of the puts and the positions in $S$ and $B$) at maturity $T$. We also determine the distribution corresponding to strategy 1).


In [ ]:
def simulate_multiple_paths_vectorized(K: float, T: float, S_0: float, mu: float, sigma: float,
                                       B_0: float, r: float, num_time_steps_per_unit_of_time: int,
                                       num_puts: int, M: int):
    """
    Vectorized Monte Carlo simulation for delta hedging strategy.
    
    Parameters:
    -----------
    K : float
        Strike price of the put option
    T : float
        Time to maturity
    S_0 : float
        Initial stock price
    mu : float
        Drift parameter for stock price
    sigma : float
        Volatility parameter for stock price
    B_0 : float
        Initial value of money market account
    r : float
        Risk-free rate
    num_time_steps_per_unit_of_time : int
        Number of rebalancing steps per unit time
    num_puts : int
        Number of put options (negative = short position)
    M : int
        Number of Monte Carlo simulations
    
    Returns:
    --------
    dict : Dictionary containing terminal values for both strategies and asset prices
    """
    num_time_steps_total = int(T * num_time_steps_per_unit_of_time)
    time_delta = T / num_time_steps_total
    put = BlackScholesOptionPrice(K, r, sigma)
    
    # Initialize arrays for M simulations
    S = np.zeros((M, num_time_steps_total + 1))
    S[:, 0] = S_0
    B = np.zeros((M, num_time_steps_total + 1))
    B[:, 0] = B_0
    
    phi = np.zeros((M, num_time_steps_total + 1))
    psi = np.zeros((M, num_time_steps_total + 1))
    price_puts = np.zeros((M, num_time_steps_total + 1))
    
    # Initialize portfolio using vectorized helper function
    phi[:, 0], psi[:, 0], price_puts[:, 0] = initialize_portfolio_vectorized(
        S_0, T, K, r, sigma, num_puts, B_0, M, put
    )
    
    # Generate all random numbers at once
    Z = norm.rvs(size=(M, num_time_steps_total))
    
    # Simulate paths
    for k in range(1, num_time_steps_total + 1):
        B[:, k] = B[:, k-1] * np.exp(r * time_delta)
        S[:, k] = S[:, k-1] * np.exp((mu - 0.5 * sigma**2) * time_delta + 
                                      sigma * np.sqrt(time_delta) * Z[:, k-1])
        
        if k == num_time_steps_total:
            # At maturity
            value = phi[:, k-1] * S[:, k] + psi[:, k-1] * B[:, k]
            price_puts[:, k] = num_puts * np.maximum(K - S[:, k], 0)
            total_portfolio_value = price_puts[:, k] + value
            break
        
        # Rebalance portfolio using vectorized helper function
        time_to_maturity = T - k * time_delta
        phi[:, k], psi[:, k], price_puts[:, k], value = rebalance_portfolio_vectorized(
            S[:, k], B[:, k], time_to_maturity, phi[:, k-1], psi[:, k-1], num_puts, put
        )
    
    strategy_no_risk_management = price_puts[:, -1] - price_puts[:, 0] * np.exp(r * T)
    
    return {
        "S_T": S[:, -1],
        "C_T": price_puts[:, -1] / num_puts,
        "no_hedge": strategy_no_risk_management,
        "delta_hedge": total_portfolio_value,
        "initial_put_value": price_puts[0, 0]
    }


In [ ]:
def print_strategy_comparison(results: dict, num_puts: int):
    """Print comprehensive comparison of strategies in a side-by-side table."""
    print("=" * 90)
    print(f"HEDGING STRATEGY COMPARISON ({len(results['no_hedge'])} simulations)")
    print("=" * 90)
    print(f"\nInitial Put Portfolio Value: €{results['initial_put_value']:,.2f}")
    print(f"Number of Puts: {num_puts}")
    
    # Prepare data for side-by-side comparison
    no_hedge = results['no_hedge']
    delta_hedge = results['delta_hedge']
    
    print("\n" + "-" * 90)
    print(f"{'Metric':<25} {'No Risk Management':>30} {'Delta Hedging':>30}")
    print("-" * 90)
    
    metrics = [
        ("Mean P&L", no_hedge.mean(), delta_hedge.mean()),
        ("Std Dev", no_hedge.std(), delta_hedge.std()),
        ("5% VaR", np.quantile(no_hedge, 0.05), np.quantile(delta_hedge, 0.05)),
        ("95% VaR", np.quantile(no_hedge, 0.95), np.quantile(delta_hedge, 0.95)),
        ("Min", no_hedge.min(), delta_hedge.min()),
        ("Max", no_hedge.max(), delta_hedge.max()),
    ]
    
    for name, val1, val2 in metrics:
        print(f"{name:<25} {val1:>29,.2f}€ {val2:>29,.2f}€")
    
    # Calculate percentage of losses
    pct_loss_no_hedge = (no_hedge < 0).mean() * 100
    pct_loss_delta = (delta_hedge < 0).mean() * 100
    print(f"{'% of losses':<25} {pct_loss_no_hedge:>29.1f}% {pct_loss_delta:>29.1f}%")
    print("-" * 90)


In [ ]:
M = 50_000  # number of Monte Carlo replications
print(f"Running {M} Monte Carlo simulations with vectorized implementation...")
results = simulate_multiple_paths_vectorized(K, T, S_0, mu, sigma, B_0, r, 
                                             num_time_steps_per_unit_of_time, num_puts, M)

# Extract results
sims_S = results["S_T"]
sims_C = results["C_T"]
strategy_no_risk_management = results["no_hedge"]
strategy_delta_hedge = results["delta_hedge"]


In [ ]:
# Print comprehensive comparison
print_strategy_comparison(results, num_puts)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(25, 7))
pd.DataFrame(sims_S).plot(kind="density", ax=axs[0], title="Distribution $S_T$")
pd.DataFrame(sims_C).plot(kind="hist", ax=axs[1], title="Distribution $C_T$", bins=25, density=True)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(25, 7))
pd.DataFrame(strategy_no_risk_management).plot(kind="hist", ax=axs[0], title="Distribution net cashflow at maturity for strategy 1 (no active risk management", bins=50, density=True)
pd.DataFrame(strategy_delta_hedge).plot(kind="hist", ax=axs[1], title="Distribution net cashflow at maturity for discrete-time delta-hedging strategy", bins=50, density=True)
print(f"5% quantile no risk management: {round(np.quantile(strategy_no_risk_management, 0.05))}")
print(f"5% quantile delta hedging with frequency={num_time_steps_per_unit_of_time} per year: {round(np.quantile(strategy_delta_hedge, 0.05))}")

## 2. Understanding Hedging Error

The delta-hedging strategy shows non-zero P&L at maturity due to discrete rebalancing.

Let us investigate impact of rebalancing frequency.

In [ ]:
# Analyze impact of rebalancing frequency
frequencies = [12, 52, 252, 504]  # monthly, weekly, daily, twice-daily
M_sensitivity = 10_000
results_by_freq = {}

print("Running sensitivity analysis...")
for freq in frequencies:
    print(f"  Frequency: {freq} rebalances per year...")
    results_by_freq[freq] = simulate_multiple_paths_vectorized(
        K, T, S_0, mu, sigma, B_0, r, freq, num_puts, M_sensitivity
    )
print("Done!")

In [ ]:
# Plot VaR vs rebalancing frequency
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

vars_5pct = [np.quantile(results_by_freq[f]['delta_hedge'], 0.05) for f in frequencies]
stds = [results_by_freq[f]['delta_hedge'].std() for f in frequencies]
means = [results_by_freq[f]['delta_hedge'].mean() for f in frequencies]

axes[0].plot(frequencies, vars_5pct, marker='o', linewidth=2, markersize=8, color='red')
axes[0].set_xlabel('Rebalancing Frequency (times per year)', fontsize=11)
axes[0].set_ylabel('5% VaR of Delta Hedge P&L (€)', fontsize=11)
axes[0].set_title('Hedging Error (5% VaR) vs Rebalancing Frequency', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')

axes[1].plot(frequencies, stds, marker='s', linewidth=2, markersize=8, color='blue')
axes[1].set_xlabel('Rebalancing Frequency (times per year)', fontsize=11)
axes[1].set_ylabel('Std Dev of Delta Hedge P&L (€)', fontsize=11)
axes[1].set_title('Hedging Error (Std Dev) vs Rebalancing Frequency', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_xscale('log')

axes[2].plot(frequencies, means, marker='^', linewidth=2, markersize=8, color='green')
axes[2].set_xlabel('Rebalancing Frequency (times per year)', fontsize=11)
axes[2].set_ylabel('Mean P&L (€)', fontsize=11)
axes[2].set_title('Mean P&L vs Rebalancing Frequency', fontsize=12)
axes[2].grid(True, alpha=0.3)
axes[2].set_xscale('log')
axes[2].axhline(0, color='black', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


## 3. Delta-Gamma Hedging

While delta hedging neutralizes first-order price risk, it leaves the portfolio exposed to **gamma risk**. Delta-gamma hedging addresses this by making the portfolio both delta-neutral and gamma-neutral.

### Strategy Overview

To achieve delta-gamma neutrality, we need **two hedging instruments**:
1. The underlying asset $S$ (which has gamma = 0)
2. Another option (e.g., a call option) with non-zero gamma

The portfolio consists of:
- $a$ put options (the position we want to hedge)
- $\phi$ units of stock $S$
- $\xi$ units of a hedging call option
- $\psi$ units of the money market account $B$

At each rebalancing time, we solve:
- **Delta neutrality**: $a \cdot \Delta_{\text{put}} + \phi + \xi \cdot \Delta_{\text{call}} = 0$
- **Gamma neutrality**: $a \cdot \Gamma_{\text{put}} + \xi \cdot \Gamma_{\text{call}} = 0$
- **Budget neutrality**: Value of $(S, B, \text{call})$-portfolio remains constant

From gamma neutrality: $\xi = -a \cdot \frac{\Gamma_{\text{put}}}{\Gamma_{\text{call}}}$

From delta neutrality: $\phi = -a \cdot \Delta_{\text{put}} - \xi \cdot \Delta_{\text{call}}$

### Delta-Gamma Hedging Parameters

We'll use a call option with a different strike as the hedging instrument.

In [ ]:
# Hedging call option parameters (different strike than the put)
K_hedge = 110

### Delta-Gamma Hedging Implementation

In [ ]:
def simulate_delta_gamma_hedging(K_put: float, K_call: float, T: float, S_0: float, 
                                  mu: float, sigma: float, B_0: float, r: float, 
                                  num_time_steps_per_unit_of_time: int, num_puts: int, M: int):
    """
    Vectorized Monte Carlo simulation for delta-gamma hedging strategy.
    
    Uses a call option as additional hedging instrument to neutralize gamma risk.
    
    Returns:
    --------
    dict : Dictionary containing terminal values for delta-gamma hedged portfolio
    """
    num_time_steps_total = int(T * num_time_steps_per_unit_of_time)
    time_delta = T / num_time_steps_total
    
    put = BlackScholesOptionPrice(K_put, r, sigma)
    call_hedge = BlackScholesOptionPrice(K_call, r, sigma)
    
    # Initialize arrays
    S = np.zeros((M, num_time_steps_total + 1))
    S[:, 0] = S_0
    B = np.zeros((M, num_time_steps_total + 1))
    B[:, 0] = B_0
    
    phi = np.zeros((M, num_time_steps_total + 1))  # stock position
    xi = np.zeros((M, num_time_steps_total + 1))   # hedging call position (number of calls)
    psi = np.zeros((M, num_time_steps_total + 1))  # money market position
    price_puts = np.zeros((M, num_time_steps_total + 1))
    
    # Initial positions
    put_price_initial = put.price_put(current_stock_price=S_0, time_to_maturity=T)
    call_price_initial = call_hedge.price_call(current_stock_price=S_0, time_to_maturity=T)
    price_puts[:, 0] = num_puts * put_price_initial
        
    gamma_put_0 = put.gamma_put(current_stock_price=S_0, time_to_maturity=T)
    gamma_call_0 = call_hedge.gamma_call(current_stock_price=S_0, time_to_maturity=T)
    xi[:, 0] = -num_puts * gamma_put_0 / gamma_call_0 # gamma neutrality
    
    delta_put_0 = put.delta_put(current_stock_price=S_0, time_to_maturity=T)
    delta_call_0 = call_hedge.delta_call(current_stock_price=S_0, time_to_maturity=T)
    phi[:, 0] = -num_puts * delta_put_0 - xi[:, 0] * delta_call_0 # delta neutrality
    
    # Budget neutrality: total portfolio value = 0 at t=0
    call_position_value_0 = xi[:, 0] * call_price_initial
    psi[:, 0] = -(price_puts[:, 0] + call_position_value_0 + phi[:, 0] * S[:, 0]) / B[:, 0]
    
    # Generate all random numbers
    Z = norm.rvs(size=(M, num_time_steps_total))
    
    # Simulate paths
    for k in range(1, num_time_steps_total + 1):
        B[:, k] = B[:, k-1] * np.exp(r * time_delta)
        S[:, k] = S[:, k-1] * np.exp((mu - 0.5 * sigma**2) * time_delta + 
                                      sigma * np.sqrt(time_delta) * Z[:, k-1])
        
        time_to_maturity = T - k * time_delta
        
        # Mark to market: calculate current value of call position from previous period
        call_price_at_k = call_hedge.price_call(current_stock_price=S[:, k], 
                                                 time_to_maturity=time_to_maturity)
        call_position_value_carried = xi[:, k-1] * call_price_at_k
        
        # Calculate total value of hedging portfolio (carried forward from previous period)
        value = phi[:, k-1] * S[:, k] + call_position_value_carried + psi[:, k-1] * B[:, k]
        
        if k == num_time_steps_total:
            # At maturity: options have intrinsic value only
            price_puts[:, k] = num_puts * np.maximum(K_put - S[:, k], 0)
            call_position_value_final = xi[:, k-1] * np.maximum(S[:, k] - K_call, 0)
            total_portfolio_value = price_puts[:, k] + phi[:, k-1] * S[:, k] + call_position_value_final + psi[:, k-1] * B[:, k]
            break
        
        price_puts[:, k] = num_puts * put.price_put(current_stock_price=S[:, k], 
                                                     time_to_maturity=time_to_maturity)
        
        # Rebalance: gamma neutrality then delta neutrality
        gamma_put_k = put.gamma_put(current_stock_price=S[:, k], time_to_maturity=time_to_maturity)
        gamma_call_k = call_hedge.gamma_call(current_stock_price=S[:, k], time_to_maturity=time_to_maturity)
        
        # Handle division by zero safely: when gamma_call is effectively zero, fall back to delta-only hedging (xi=0)
        epsilon = 1e-10
        with np.errstate(divide="ignore", invalid="ignore"):
            xi_temp = -num_puts * gamma_put_k / gamma_call_k
        # Replace inf/nan with 0
        xi[:, k] = np.where(np.abs(gamma_call_k) > epsilon, xi_temp, 0.0)
        
        delta_put_k = put.delta_put(current_stock_price=S[:, k], time_to_maturity=time_to_maturity)
        delta_call_k = call_hedge.delta_call(current_stock_price=S[:, k], time_to_maturity=time_to_maturity)
        phi[:, k] = -num_puts * delta_put_k - xi[:, k] * delta_call_k
        
        # Budget neutrality: new portfolio value = carried forward value
        call_position_value_new = xi[:, k] * call_price_at_k
        psi[:, k] = (value - phi[:, k] * S[:, k] - call_position_value_new) / B[:, k]
    
    return {
        "S_T": S[:, -1],
        "delta_gamma_hedge": total_portfolio_value,
        "initial_put_value": price_puts[0, 0]
    }


### Run Delta-Gamma Hedging Simulations

In [ ]:
print("Running delta-gamma hedging simulations...")
results_dg = simulate_delta_gamma_hedging(K, K_hedge, T, S_0, mu, sigma, B_0, r, 
                                          num_time_steps_per_unit_of_time, num_puts, M)
print("Done!")

### Compare Delta vs Delta-Gamma Hedging

In [ ]:
# Comparison table
print("=" * 90)
print("COMPARISON: Delta Hedging vs Delta-Gamma Hedging")
print("=" * 90)
print(f"\nNumber of simulations: {M}")
print(f"Rebalancing frequency: {num_time_steps_per_unit_of_time} times per year")
print(f"Put strike: €{K}")
print(f"Hedging call strike: €{K_hedge}")

strategies_comparison = [
    ('Delta Hedging Only', strategy_delta_hedge),
    ('Delta-Gamma Hedging', results_dg['delta_gamma_hedge'])
]

print("\n" + "-" * 90)
print(f"{'Strategy':<30} {'Mean P&L':>15} {'Std Dev':>15} {'5% VaR':>15} {'% Losses':>10}")
print("-" * 90)

for name, data in strategies_comparison:
    print(f"{name:<30} €{data.mean():>14,.2f} €{data.std():>14,.2f} €{np.quantile(data, 0.05):>14,.2f} {(data < 0).mean() * 100:>9.1f}%")

print("-" * 90)
print(f"\nStd Dev Reduction: {(1 - results_dg['delta_gamma_hedge'].std() / strategy_delta_hedge.std()) * 100:.1f}%")
print(f"5% VaR Improvement: {np.quantile(strategy_delta_hedge, 0.05) - np.quantile(results_dg['delta_gamma_hedge'], 0.05):.2f}")


In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Distribution comparison
axes[0].hist(strategy_delta_hedge, bins=50, density=True, alpha=0.6, 
             label='Delta Hedging', edgecolor='black', color='blue')
axes[0].hist(results_dg['delta_gamma_hedge'], bins=50, density=True, alpha=0.6, 
             label='Delta-Gamma Hedging', edgecolor='black', color='green')
axes[0].set_xlabel('Net P&L at Maturity (€)', fontsize=11)
axes[0].set_ylabel('Density', fontsize=11)
axes[0].set_title('Distribution Comparison', fontsize=12)
axes[0].axvline(0, color='red', linestyle='--', linewidth=1)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot comparison
data_for_box = [strategy_delta_hedge, results_dg['delta_gamma_hedge']]
axes[1].boxplot(data_for_box, labels=['Delta\nHedging', 'Delta-Gamma\nHedging'])
axes[1].set_ylabel('Net P&L at Maturity (€)', fontsize=11)
axes[1].set_title('Box Plot Comparison', fontsize=12)
axes[1].axhline(0, color='red', linestyle='--', linewidth=1)
axes[1].grid(True, alpha=0.3, axis='y')

# Risk metrics comparison
metrics = ['Mean', 'Std Dev', '5% VaR', 'Max Loss']
delta_metrics = [
    strategy_delta_hedge.mean(),
    strategy_delta_hedge.std(),
    np.quantile(strategy_delta_hedge, 0.05),
    strategy_delta_hedge.min()
]
delta_gamma_metrics = [
    results_dg['delta_gamma_hedge'].mean(),
    results_dg['delta_gamma_hedge'].std(),
    np.quantile(results_dg['delta_gamma_hedge'], 0.05),
    results_dg['delta_gamma_hedge'].min()
]

x = np.arange(len(metrics))
width = 0.35

axes[2].bar(x - width/2, delta_metrics, width, label='Delta Hedging', alpha=0.8)
axes[2].bar(x + width/2, delta_gamma_metrics, width, label='Delta-Gamma Hedging', alpha=0.8)
axes[2].set_ylabel('Value (€)', fontsize=11)
axes[2].set_title('Risk Metrics Comparison', fontsize=12)
axes[2].set_xticks(x)
axes[2].set_xticklabels(metrics, rotation=15, ha='right')
axes[2].legend()
axes[2].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()
